Maybe try filtering out "proposed"

In [1]:
import numpy as np
import pandas as pd
import os
import dill
from utils.case_directory import CaseDirectory
from utils.case_metadata import CaseMetadata
from extractors.jury_ruling_classifier import JuryRulingClassifier
from extractors.bench_ruling_classifier import BenchRulingClassifier

In [32]:
df = pd.read_csv("out/state_metadata.csv")

In [33]:
df

,Unnamed: 0,court,title,docket,judges,judge,type,link,status,flags,nature_of_suit,cause,magistrate,metadata_path,trial,trial_type,trial_result
0,0,"Oregon State, Multnomah County, Circuit Court",State of Oregon vs Abed Alkader Fattoum,18CR08309,"['Angel Lopez', 'Angela Lucero', 'Benjamin Sou...",NaN,Offense Felony,https://www.docketalarm.com/cases/Oregon_State...,Closed,NaN,NaN,NaN,NaN,100_random_sample/Oregon_State_Multnomah_Count...,False,unknown,NaN
1,1,"Texas State, Harris County, 152nd District Court","CASARES, DOUGLAS vs. CASARES, ROSA",202236481,['ROBERT K. SCHAFFER'],ROBERT K. SCHAFFER,Motor Vehicle Accident,https://www.docketalarm.com/cases/Texas_State_...,Disposed (Final),NaN,NaN,NaN,NaN,100_random_sample/Texas_State_Harris_County_15...,False,unknown,NaN
2,2,"Texas State, Dallas County, 101st District Court",ANDREW HOWARD vs. JIMMY SIMPSON,DC-19-03669,['STACI WILLIAMS'],STACI WILLIAMS,MOTOR VEHICLE ACCIDENT,https://www.docketalarm.com/cases/Texas_State_...,CLOSED,NaN,NaN,NaN,NaN,100_random_sample/Texas_State_Dallas_County_10...,False,unknown,NaN
3,3,"Massachusetts State, Superior Court, Barnstabl...","Sprague, Dawn vs. Corner Cycle Of Cape Cod, Inc.",2072CV00184,NaN,NaN,Torts,https://www.docketalarm.com/cases/Massachusett...,Open,NaN,NaN,NaN,NaN,100_random_sample/Massachusetts_State_Superior...,False,unknown,NaN
4,4,"Texas State, Dallas County, 116th District Court",DESOTO APARTMENTS LTD vs. DALLAS CENTRAL APPRA...,DC-13-11287,['TONYA PARKER'],TONYA PARKER,TAX APPRAISAL,https://www.docketalarm.com/cases/Texas_State_...,CLOSED,NaN,NaN,NaN,NaN,100_random_sample/Texas_State_Dallas_County_11...,False,unknown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,102,"Connecticut State, Superior Court","MACCHIAROLI, ANTHONY v. OFFICE OF CLAIMS COMIS...",LLI-CV23-5015421-S,['ANDREW RORABACK'],ANDREW RORABACK,M90 - Misc - All other,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...,False,unknown,NaN
103,103,"Connecticut State, Superior Court","DAO, THAO v. ST. LAURENT, GERALD",HHD-CV17-6080452-S,"['CONSTANCE EPSTEIN', 'DAVID SHERIDAN', 'SUSAN...",NaN,V01 - Vehicular - Motor Vehicles - Driver and/...,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...,False,unknown,NaN
104,104,"Connecticut State, Superior Court","RENFORS, TARYN v. BRANFORD MANOR PRESERVATION....",KNL-CV21-6051823-S,['KAREN GOODROW'],KAREN GOODROW,T03 - Torts - Defective Premises - Private - O...,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...,False,unknown,NaN
105,105,"Connecticut State, Superior Court","ANDINO, GIRELYS Et Al v. KAISER, MAYA Et Al",UWY-CV22-6068059-S,['ROBERT D ANDREA'],ROBERT D ANDREA,V01 - Vehicular - Motor Vehicles - Driver and/...,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...,False,unknown,NaN


In [28]:
df = df.drop("Unnamed: 0", axis=1)

In [22]:
df.path = df.path.apply(lambda s: s.split("../")[1])

In [29]:
df.to_csv("tests/fed_key.csv", index=False)

In [7]:
with open("llama31_key_new_prompt1.p", "wb") as f:
    dill.dump(df, f)

'100_random_fed/Texas_Eastern_District_Court/2--21-cv-00084/The_Chamberlain_Group_LLC_v._Overhead_Door_Corporation_et_al/600.txt'

In [19]:
print(logs[0].document_classification["chunks_after_semantic_filter"][0])

. 59 59 59 
59 60 61 61 61 61 61 61) On January 2 202 the jury returned a verdict finding 


Case 2cv-0008JRG   Document 65    Page of PageID #:  3163that Defendants infringed Claims and 2of the ’40patent, that Defendants’ infringement was 
not willful, and that the Asserted Claims are not invalid as obvious or for improper inventorship


In [9]:
logs[0].document_classification["response"]

'{\n"reasoning": "According to the documents, the jury returned a verdict finding that defendants infringed Claims 1 and 2 of the \'40 patent, and the asserted claims are not invalid as obvious or for improper inventorship. This shows that the jury ruled in favor of the plaintiff because it found that the defendants\' actions constituted infringement.",\n"category": "plaintiff"\n}'

In [22]:
index = 6
log = logs[index]

In [31]:
log.metadata_classification.keys()

dict_keys(['docs_before_chunking', 'chunks', 'chunks_after_keyword_filter', 'chunks_after_semantic_filter', 'llm_context', 'llm_response_raw', 'llm_response'])

In [11]:
log.config["language_model_prompt"]

'You are an expert legal analyst. You will be given a list of excerpts from legal documents relating to a case in the United States with a decision made by a jury.\n        Documents are separated by ||. All documents correspond to the same case. Classify the outcome of this case into one of the following categories:\n\n        plaintiff\n        defendant\n        undetermined\n\n        If the jury decided in favor of the plaintiff, classify the outcome as plaintiff. \n        The documents may refer to the plaintiff as "plaintiff" or by name. \n        Here is a list of plaintiff names: Bennion and Deville Fine Homes Inc, Windermere Services Southern California Inc, Bennion and Deville Fine Homes SoCal, Inc.\n        If the jury decided in favor of the defendant, classify the outcome as defendant.\n        The documents may refer to the defendant as "defendant" or by name.\n        Here is a list of defendant names: Windermere Real Estate Services Company, Does, Robert L Bennion, Jo

In [23]:
print(log.metadata_classification["llm_response_raw"])
print(log.metadata_classification["llm_context"])

{
"reasoning": "According to the documents, an order was entered granting summary judgment in favor of the defendant on Count 3 and denying it as to Counts 1 and 2. This shows that the jury ruled in favor of the defendant because the court's decision granted the defendant's motion for summary judgment.",
"category": "defendant"
}
MOTION Leave to Conduct Post-Verdict Juror Interviews  with Exhibits and Certificate of Service, MOTION for Order by Defendant Goodyear Dunlop Tires North America Ltd. Responses due by 9/17/2007 (Moor, Edward) (Entered: 08/29/2007)||Proposed Jury Instructions by Goodyear Tire & Rubber Company, The. (Attachments: # 1 Supplement -Cert of Serv of Exh H to Prop Final Pretrial Ord, Defs Prop Jury Inst - Corrected)(Moor, Edward) (Entered: 06/08/2007)||MOTION for Summary Judgment and good faith settlement finding by Counter Defendant William Booker. Responses due by 8/26/2005 (Attachments: # 1 # 2)(Januszyk, Ximen) (Entered: 08/05/2005)||EDIT BY CLERK-DOCUMENTS STRIC

In [21]:
print(log.document_classification["llm_response_raw"])
print(log.document_classification["llm_context"])

{
"reasoning": "According to the documents, the jury ruled in favor of Sumotext Corp. because it is listed as 'Yes' on a verdict form.",
"category": "plaintiff"
}
.  
I'M SORRY YOU DIDN'T READ MY STANDING ORDER, BUT I REQUIRE 
THAT THERE ONLY BE ONE INSTRUCTION PER PAGE.  
SO PLAINTIFF'S -- IF IT'S DISPUTED, PLAINTIFF'S NUMBER 1, 
THAT TAKES A PAGE, AND IT HAS ALL OF PLAINTIFF'S ARGUMENT.  
THEN IF THE DEFENDANT HAS AN ALTERNATE, IT'S A SEPARATE 
PAGE, DEFENDANTS' DISPUTED NUMBER 1.  
AND THAT WAY -- I MEAN, FIRST OF ALL, I CAN PULL THEM 
APART AND WORK ON ONE OR THE OTHER.  I SIMPLY CAN'T -- I KNOW 
SOME JUDGES RUN THEM TOGETHER, AND I DON'T.  SO THAT NEEDS TO 
BE FIGURED OUT AS WELL SO THAT I CAN WORK WITH THAT.  
I THINK THAT'S PLENTY.  
SO MY OVERALL IMPRESSION IS THAT THEY APPEAR 
ARGUMENTATIVE, AND I NEED YOU TO REALLY THINK ABOUT THE TONE||.
MR. KESSELMAN:  I GUESS I WAS LOOKING AT YOUR RULES.  
OKAY.  THANK YOU, YOUR HONOR.  
MR. BLOCH:  IN ANY EVENT, YOUR HONOR, I WOULD LIKE T

In [26]:
print(log.config["language_model_prompt"])


        You are an expert legal analyst. You will be given a list of excerpts from legal documents relating to a case in the United States with a decision made by a jury. Documents are separated by ||. All documents correspond to the same case. Classify the outcome of this case into one of the following categories:

        plaintiff
        defendant
        undetermined

        If the jury decided in favor of the plaintiff, classify the outcome as plaintiff. 
        The documents may refer to the plaintiff as "plaintiff" or by name. 
        Here is a list of plaintiff names: Trish Lee McCloud, McCloud
        If the jury decided in favor of the defendant, classify the outcome as defendant.
        The documents may refer to the defendant as "defendant" or by name.
        Here is a list of defendant names: Goodyear Dunlop Tires North America Ltd, Goodyear Tire & Rubber Company, The, William Booker, Goodyear Dunlop Tire
        If the documents provided do not identify the jury ve

In [12]:
print(key.text[index])

Case: 3:14-cv-00099-bbc   Document #: 603   Filed: 11/06/17   Page 1 of 1
IN THE UNITED STATES DISTRlCT COURT 
FOR THE WESTERN DISTRlCT OF WISCONSIN 
MARY HALEY and MICHAEL HALEY, 
LESLIE BANKS and JAMES HAL BANKS, · 
ANNIE BUINEWICZ and BRlAN BUINEWICZ, 
TERRANCE McIVER AND JEAN ANN McIVER, 
SUSAN SENYI(, CHRlSTIAN SENYK, 
GARY SAMUELS, PATRICIA GROOME, 
MATTHEW DELLER, RENEE DELLER 
and MARJE LOHR, on behalf of themselves and 
all others similarly situated, 
Plaintiffs, 
v. 
KOLBE &KOLBE MILLWORl( CO., INC., 
Defendant, 
and 
FIREMAN'S FUND INSURANCE COMPANY 
and UNITED STATES FIRE INSURANCE COMPANY, 
Intervenor Defendants. 
JUDGMENT IN A CIVIL CASE 
Case No. l 4-cv-99-bbc 
This action came for consideration before the court with District Judge 
Barbara B. Crabb presiding. The issues have been considered and a decision has been 
rendered. 
IT IS ORDERED AND ADJUDGED that judgment is entered in favor of defendant 
Kolbe & Kolbe Millwork Co., Inc. declaring that intervenor defendants U

### Header Filtering

Header Examples

Case: 3:14-cv-00099-bbc   Document #: 384   Filed: 11/06/15   Page 3 of 6
Case 2:21-cv-00084-JRG   Document 353   Filed 03/11/22   Page 2 of 8 PageID #:  12877
1:04-cv-01118-JBM-JAG   # 93    Page 2 of 3                                              
Case 5:16-cv-01370-BLF   Document 524   Filed 12/30/20   Page 20 of 44
1:04-cv-01118-JBM-JAG   # 386    Page 2 of 2                                             

In [28]:
import regex as re

In [30]:
print(key.text[2])

Case 5:16-cv-01370-BLF Document 470 Filed 03/06/20 Page 1 of 10
UNITED STATES DISTRICT COURT
NORTHERN DISTRICT OF CALIFORNIA
SAN JOSE DIVISION
SUMOTEXT CORR, a Nevada corporation,
Case No.: 16-CV-01370-BLF
Sumotext,
VERDICT FORM
vs.
ZOOVE, INC; et a1.
Hon. Beth Labsan Freeman
Defendants.
 
 
 
10
11
12
13
14
15
16
17
18
19
2O
21 ~.
22
23
24
25
26
27
28
 
 
 
 
 


Case 5:16-cv-01370-BLF Document 470 Filed 03/06/20 Page 2 of 10
1 We, the Jury, unanimously find as follows:
I. QUESTIONS APPLICABLE TO LEASING MARKET CLAIMS
A. MARKET DEFINITION
1.
Did Sumotext prove by a preponderance of the evidence a relevant market for
leasing ** numbers in the United States?
Yes __ (‘5Yes” is a finding for Sumotext)
No X (“No” is a finding for Defendants)
If you answered “Yes” to Question 1, proceed to Question 2.
'
 
If you answered “No” to Question 1, you have found no liability for Sumotext’s Leasing claims. Do not
answer any other questions in Section 1. Please proceed to Section II (Question 8).


In [31]:
text = key.text[2]
extra_numbers = "(\d[\s|\W])+"
case_number = "(Case|case)\s?:\s+?\S+"
document_number = "(Document|Document|doc|Doc)\s*?#?:\s*?\d+"
filing_date = "(Filed|filed)\s*?:*?\s+\d+\/\d+\/\d+"
page_number = "(Page|page)\s*?\d+\s*(of\s*?\d+)?"
for seq in [extra_numbers, "Filed (\d+\/)*\d+"]:
    text = re.sub(seq, "", text)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\d'
/var/folders/4d/7fpy9t817xb0g1fjs74tbys00000gn/T/ipykernel_33855/755875549.py:2: SyntaxWarning: invalid escape sequence '\d'
  extra_numbers = "(\d[\s|\W])+"
/var/folders/4d/7fpy9t817xb0g1fjs74tbys00000gn/T/ipykernel_33855/755875549.py:3: SyntaxWarning: invalid escape sequence '\s'
  case_number = "(Case|case)\s?:\s+?\S+"
/var/folders/4d/7fpy9t817xb0g1fjs74tbys00000gn/T/ipykernel_33855/755875549.p

In [32]:
print(text)

Case 1cv-0137BLF Document 47Page of 1UNITED STATES DISTRICT COURT
NORTHERN DISTRICT OF CALIFORNIA
SAN JOSE DIVISION
SUMOTEXT CORR, a Nevada corporation,
Case No.: 1CV-0137BLF
Sumotext,
VERDICT FORM
vs.
ZOOVE, INC; et a
Hon. Beth Labsan Freeman
Defendants.
 
 
 
11111111112O
2~.
2222222 
 
 
 
 


Case 1cv-0137BLF Document 47Page of 1We, the Jury, unanimously find as follows:
I. QUESTIONS APPLICABLE TO LEASING MARKET CLAIMS
A. MARKET DEFINITION

Did Sumotext prove by a preponderance of the evidence a relevant market for
leasing ** numbers in the United States?
Yes __ (‘5Yes” is a finding for Sumotext)
No X (“No” is a finding for Defendants)
If you answered “Yes” to Question  proceed to Question 
'
 
If you answered “No” to Question  you have found no liability for Sumotext’s Leasing claims. Do not
answer any other questions in Section  Please proceed to Section II (Question .
B. CLAIM  Sherman Act §  Agreement to Restrain Trade in the Leasing Market -
'Against VHT and StarSteve
p

‘
,

## Manually Testing Classification

In [25]:
import ollama

In [38]:
df = dill.load(open("llama31_key_new_prompt1.p", "rb"))
log_list = dill.load(open("llama31_logs_new_prompt1.p", "rb"))

In [39]:
df

,path,text,category,metadata_path,result
0,100_random_fed/Texas_Eastern_District_Court/2-...,Case 2:21-cv-00084-JRG Document 600 Filed ...,plaintiff,100_random_fed/Texas_Eastern_District_Court/2-...,plaintiff
1,100_random_fed/Wisconsin_Western_District_Cour...,Case: 3:14-cv-00099-bbc Document #: 603 Fi...,defendant,100_random_fed/Wisconsin_Western_District_Cour...,undetermined
2,100_random_fed/California_Northern_District_Co...,Case 5:16-cv-01370-BLF Document 470 Filed 03/0...,defendant,100_random_fed/California_Northern_District_Co...,undetermined
3,100_random_fed/New_Jersey_District_Court/1--16...,Case 1:16-cv-00395-JBS-JS Document 234 Fil...,defendant,100_random_fed/New_Jersey_District_Court/1--16...,defendant
4,100_random_fed/California_Central_District_Cou...,Case 2:06-cv-03923-BRO-FFM Document 311 Fi...,defendant,100_random_fed/California_Central_District_Cou...,defendant
5,100_random_fed/California_Central_District_Cou...,Case 5:15-cv-01921-DFM Document 270 Filed ...,defendant,100_random_fed/California_Central_District_Cou...,plaintiff
6,100_random_fed/Illinois_Central_District_Court...,1:04-cv-01118-JBM-JAG # 93 Page 1 of 3 ...,plaintiff,100_random_fed/Illinois_Central_District_Court...,plaintiff


In [47]:
key

,path,text,category,metadata_path,result
0,100_random_fed/Texas_Eastern_District_Court/2-...,Case 2:21-cv-00084-JRG Document 600 Filed ...,plaintiff,100_random_fed/Texas_Eastern_District_Court/2-...,undetermined
1,100_random_fed/Wisconsin_Western_District_Cour...,Case: 3:14-cv-00099-bbc Document #: 603 Fi...,defendant,100_random_fed/Wisconsin_Western_District_Cour...,defendant
2,100_random_fed/California_Northern_District_Co...,Case 5:16-cv-01370-BLF Document 470 Filed 03/0...,defendant,100_random_fed/California_Northern_District_Co...,defendant
3,100_random_fed/New_Jersey_District_Court/1--16...,Case 1:16-cv-00395-JBS-JS Document 234 Fil...,defendant,100_random_fed/New_Jersey_District_Court/1--16...,defendant
4,100_random_fed/California_Central_District_Cou...,Case 2:06-cv-03923-BRO-FFM Document 311 Fi...,defendant,100_random_fed/California_Central_District_Cou...,defendant
5,100_random_fed/California_Central_District_Cou...,Case 5:15-cv-01921-DFM Document 270 Filed ...,defendant,100_random_fed/California_Central_District_Cou...,plaintiff
6,100_random_fed/Illinois_Central_District_Court...,1:04-cv-01118-JBM-JAG # 93 Page 1 of 3 ...,plaintiff,100_random_fed/Illinois_Central_District_Court...,plaintiff


In [45]:
log

[ExtractorLog(config={'language_model_prompt': '\n        You are an expert legal analyst. You will be given a list of excerpts from legal documents relating to a case in the United States with a decision made by a jury. Documents are separated by ||. All documents correspond to the same case. Classify the outcome of this case into one of the following categories:\n\n        plaintiff\n        defendant\n        undetermined\n\n        If the jury decided in favor of the plaintiff, classify the outcome as plaintiff. \n        The documents may refer to the plaintiff as "plaintiff" or by name. \n        Here is a list of plaintiff names: The Chamberlain Group, Inc., The Chamberlain Group LLC\n        If the jury decided in favor of the defendant, classify the outcome as defendant.\n        The documents may refer to the defendant as "defendant" or by name.\n        Here is a list of defendant names: Overhead Door Corporation, GMI Holdings Inc.\n        If the documents provided do not i

In [26]:
prompt, docs = logs[0].config["language_model_prompt"], logs[0].document_classification["llm_context"]

In [27]:
response = ollama.generate(
            model="llama3",
            prompt=docs,
            system=prompt,
            format="json",
        )

In [29]:
response["response"]

'{\n"reasoning": "According to the documents, the jury returned a verdict finding that defendants infringed Claims 4 and 20 of the \'404 patent. This shows that the jury ruled in favor of plaintiff because they found that the defendants infringed on the plaintiff\'s patent.",\n"category": "plaintiff"\n}'

In [18]:
m = CaseMetadata.from_metadata_path(key.loc[0, "metadata_path"])